<a href="https://colab.research.google.com/github/LukeCoelho/module1_lectures/blob/main/Copy_of_1_6_introduction_to_piping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [83]:
%%R
library(dplyr)

# Introduction to piping

## Review -- Piping with `dplyr`

You might be familiar with piping in R using the `%>%` operator from `dplyr`

In [84]:
%%R
pi %>% 
sqrt %>% 
round(2) %>% 
as.character

[1] "1.77"


## Making `pipeable` functions using `composable`

To get functions to be pipeable in Python, we need to wrap them in `pipeable` from the `composable` module

In [85]:
!pip install composable # Install if missing or in colab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
from composable import pipeable

## Making some pipeable functions

Before I can recreate the R example, I need to make some pipeable functions.

## Making a pipeable `sqrt`

To make an existing function pipeable, I need to wrap or *decorate* it with `pipeable`.

In [87]:
import math as m
sqrt = pipeable(m.sqrt)

In [88]:
2 >> sqrt

1.4142135623730951

## Some common functions are not actually functions

Some Python "functions" are not actually functions, but type constructors.  Examples include `str`, `float`, `int`, `list`, etc.  This also includes the most powerful type constructor of them all, `type`

In [89]:
type(str)

type

In [117]:
wont_always_work = pipeable(float)
3.5 >> sqrt >> wont_always_work

1.8708286933869707

## Use a `lambda` to create a pipeable type conversion function

To be safe, we need to wrap type constructors in a lambda, THEN `pipeable`

In [91]:
toStr = pipeable(lambda n: str(n))

In [92]:
3.5 >> toStr

'3.5'

## Piping and multiple arguments

Piped in data is inserted **on the right**

In [93]:
test = pipeable(lambda a, b: f"first:{a} second:{b}")

In [94]:
test(1,2)

'first:1 second:2'

In [95]:
1 >> test(2)

'first:2 second:1'

In [96]:
2 >> test(1)

'first:1 second:2'

## Rearranging argument order

The default `round` function uses `round(number, digits)`

In [97]:
round(m.pi, 2)

3.14

For piping, it is more convenient to switch the order.

In [98]:
rnd = pipeable(lambda d, n: round(n, d))

In [99]:
m.pi >> rnd(2) 

3.14

## Recreating the R example

In [100]:
m.pi >> sqrt >> rnd(2) >> toStr

'1.77'

In [101]:
m.pi >> 
sqrt >> 
rnd(2) >> 
toStr

SyntaxError: ignored

## Hint 1: Wrap multi-line piped expressions in parentheses

In [ ]:
(m.pi >> 
 sqrt >> 
 rnd(2) >> 
 toStr)

## Hint 2: Put the pipes at the start of a line

In [ ]:
(m.pi 
 >> sqrt 
 >> rnd(2) 
 >> toStr
)

## Pipeable functions return functions when partially complete

Note that `pipeable` functions are curried, meaning they return functions if not provided with enough arguments.

In [ ]:
threeArgs= pipeable(lambda a, b, c: f"first:{a} second:{b} third:{c}")

In [ ]:
threeArgs("Bob")

In [ ]:
threeArgs("Bob", "Alice")

## We can save and call a partial functions 

In [ ]:
bob = threeArgs("Bob")

In [ ]:
bob(2,3)

In [ ]:
bobAndAlice = bob("Alice")

In [ ]:
bobAndAlice(3)

## Example

Suppose that I round to two decimal places A LOT.  In this case it might be nice to have a specialized function

In [ ]:
rndToTwo = rnd(2)

In [ ]:
m.pi >> rndToTwo

<font color="red"><h1>Exercise 3</h1></font>

Here is a problem that you solved in a previous activity:

    The function `random` from the `random` module can be used to generate numbers between 0 and 1 at random. We want to return numbers between $a$ and $b$ at random, which can be accomplished using the formula $V = (b - a)*random() + a$.

    Write a lambda function that takes `a` and `b` as arguments are returns a number between `a` and `b` at random.
   
Note that we can name the parts the process as follows:

> b takeAway a >> times a random number >> subtract a

Let's redo this problem, but this time with piping; where we will make a pipeable function to perform each task.

In [147]:
from random import random

In [148]:
r = random()

In [152]:
takeAway = pipeable(lambda min, max: max - min)
timesRandom = pipeable(lambda r: takeAway * r)
add = pipeable(lambda min: timesRandom + min)

In [153]:
def uniform(min, max):
    return ( max
            >> takeAway(min)
            >> timesRandom
            >> add(min)
            )

In [154]:
assert 1 <= uniform(1,2) <= 2

TypeError: ignored